In [1]:
# 导入包
import numpy as np
import pandas as pd
import os, json
from pyecharts.charts import Line, Pie, Bar, Grid, Geo, Map
import pyecharts.options as opts
from pyecharts.globals import ChartType, SymbolType, ThemeType # SymbolType标点的样式


# 解决pyecharts显示空白
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK

In [2]:
pd.set_option('display.max_rows', None) # 显示所有行
pd.set_option('display.max_columns', None) # 显示所有列
# 文件根路径
filepath_cn = "./疫情数据/国内数据/"
filepath_fg = "./疫情数据/外国数据/"
filepath_visual = './部分可视化页面/'
if not os.path.exists(filepath_visual): # 先判断这个文件夹是否已存在\
        os.mkdir(filepath_visual)

In [3]:
# 格式化时间 传入pandas 的一列
def format_time(time_series):
    # 格式化时间
    time_series.drop_duplicates('日期', inplace=True, ignore_index=True) # 先去除重复的列
    start_time = str(time_series['日期'][0])
    if start_time[-2] == '.':
        start_time += '0' # 因为float型转为字符串1,2就少个0，应该是1.20，因此先把0加上
    start_time = start_time.replace('.', '/') + '/2020'
    # print(start_time)
    end_time = str(time_series['日期'][len(time_series['日期'])-1])
    if end_time[-2] == '.':
        end_time += '0'
    end_time = end_time.replace('.', '/') + '/2020'
    # print(end_time)
    time_series['日期'] = pd.date_range(start=start_time, end=end_time)
    time_series['日期'] = time_series['日期'].astype(str)
    # cn_prov_day_data['日期']

# 中国疫情地图可视化

## 国内整体疫情地图可视化

In [4]:
cn_all_data = pd.read_csv(filepath_cn+"中国疫情数据.csv", encoding='GBK')
# 为了更直观看，将列名全部改为中文
cn_all_data.columns = ['地区', '现有确诊', '累计确诊', '现有疑似', '累计死亡', '死亡率', '累计治愈', '治愈率']
cn_all_data.drop(labels=[0], axis=0, inplace=True) # 删除第一行，第一行是全国合计数据不需要显示
cn_all_data

,地区,现有确诊,累计确诊,现有疑似,累计死亡,死亡率,累计治愈,治愈率
1,香港,773,2131,0,14,0.66,1344,63.07
2,北京,87,929,0,9,0.97,833,89.67
3,新疆,64,140,0,3,2.14,73,52.14
4,上海,19,737,0,7,0.95,711,96.47
5,广东,16,1661,0,8,0.48,1637,98.56
6,内蒙古,9,250,0,1,0.40,240,96.00
7,台湾,8,455,0,7,1.54,440,96.70
8,四川,7,603,0,3,0.50,593,98.34
9,辽宁,6,165,0,2,1.21,157,95.15
10,山东,6,798,0,7,0.88,785,98.37


In [5]:
(
    Geo()
    .add_schema(maptype="china") # 使用中国地图 
    .add(
        series_name='现有确诊', 
        data_pair=[(i, j) for i, j in zip(cn_all_data['地区'], cn_all_data['现有确诊'])], # data_pair是放数据的地方，类别和属性值
        label_opts=opts.LabelOpts(is_show=False),
        type_=ChartType.EFFECT_SCATTER # 图的类型为特效散点图
    ) # 也可以隐藏地图上的文字显示，不过悬浮点上的时候还是可以显示的 
    .set_global_opts(
        title_opts=opts.TitleOpts(title="中国现有确诊"),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=False) # 视觉印象配置项 is_piecewise=True就不是连续的颜色了，而是拆分成块了
    )
).render_notebook()

f:\python\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


可以看出北京、新疆、上海和香港是现有确诊最多的地方

In [6]:
pieces=[
    {"max": 4000000, "min": 1000000, "label": ">1000000", "color": "#C71585"},
    {"max": 999999, "min": 100000, "label": "100000-999999", "color": "#B40404"},
    {"max": 99999, "min": 10000, "label": "10000-99999", "color": "#DF0101"},
    {"max": 9999, "min": 1000, "label": "1000-9999", "color": "#F78181"},
    {"max": 999, "min": 500, "label": "500-999", "color": "#F5A9A9"},
    {"max": 499, "min": 0, "label": "0-499", "color": "#AAAAAA"},
]
# 颜色块
(
    Geo()
    .add_schema(maptype="china") # 使用中国地图 
    .add(
        series_name='累计确诊', 
        data_pair=[(i, j) for i, j in zip(cn_all_data['地区'], cn_all_data['累计确诊'])], # data_pair是放数据的地方，类别和属性值
        label_opts=opts.LabelOpts(is_show=False),
        type_=ChartType.EFFECT_SCATTER # 图的类型为特效散点图
    ) # 也可以隐藏地图上的文字显示，不过悬浮点上的时候还是可以显示的 
    .set_global_opts(
        title_opts=opts.TitleOpts(title="中国累计确诊"),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True, pieces=pieces) # 使用自定义的颜色块
    )
).render_notebook()

f:\python\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


主要还是集中在湖北及周边地区，西部和西北部和西南部确诊病例很少。

In [7]:
(
    Geo()
    .add_schema(maptype="china") # 使用中国地图 
    .add(
        series_name='累计死亡', 
        data_pair=[(i, j) for i, j in zip(cn_all_data['地区'], cn_all_data['累计死亡'])], # data_pair是放数据的地方，类别和属性值
        label_opts=opts.LabelOpts(is_show=False),
        type_=ChartType.EFFECT_SCATTER # 图的类型为特效散点图
    ) # 也可以隐藏地图上的文字显示，不过悬浮点上的时候还是可以显示的 
    .set_global_opts(
        title_opts=opts.TitleOpts(title="中国累计死亡"),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=False) # 视觉印象配置项 is_piecewise=True就不是连续的颜色了，而是拆分成块了
    )
).render_notebook()

f:\python\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


由图表可知截止到7/21号西藏、青岛、宁夏、山西、江苏、澳门没有一人死亡，大部分城市死亡人数不到10人，死亡人数主要集中在湖北地区。

In [8]:
(
    Geo()
    .add_schema(maptype="china") # 使用中国地图 
    .add(
        series_name='累计治愈', 
        data_pair=[(i, j) for i, j in zip(cn_all_data['地区'], cn_all_data['累计治愈'])], # data_pair是放数据的地方，类别和属性值
        label_opts=opts.LabelOpts(is_show=False),
        type_=ChartType.EFFECT_SCATTER # 图的类型为特效散点图
    ) # 也可以隐藏地图上的文字显示，不过悬浮点上的时候还是可以显示的 
    .set_global_opts(
        title_opts=opts.TitleOpts(title="中国累计治愈"),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True, pieces=pieces) # 使用自定义的颜色块
    )
).render_notebook()

f:\python\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


通过观察中国累积确诊散点图和中国累积治愈散点图，发现两个图之间相差不大，可以观察到疫情爆发大多集中在湖北周围省以及沿海国际交流发达大城市。

In [10]:
# 各省治愈率死亡率可视化
(
    Bar(init_opts=opts.InitOpts(width='1200px'))
    .add_xaxis([i for i in cn_all_data['地区']])
    .add_yaxis(series_name='死亡率', y_axis=[i for i in cn_all_data['死亡率']], color='green', stack='stack1')
    .add_yaxis(series_name='治愈率', y_axis=[i for i in cn_all_data['治愈率']], color='grey', stack='stack1')
    .set_global_opts(
        title_opts=opts.TitleOpts(title="国内各省治愈率和死亡率"),
        tooltip_opts=opts.TooltipOpts(trigger='axis', axis_pointer_type='cross'),
        xaxis_opts=opts.AxisOpts(name='地区', axislabel_opts={"interval":"0"}),
        yaxis_opts=opts.AxisOpts(name='%', splitline_opts=opts.SplitLineOpts(is_show=True))
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
).render_notebook()

f:\python\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


## 各省具体疫情地图可视化

In [11]:
prov = input("请输入要查看的省：")
cn_prov_data = pd.read_csv(filepath_cn+f"各省具体数据/{prov}.csv", encoding='GBK')
# 删除不需要的列
if '境外输入' in cn_prov_data['name'].values.tolist():
    cn_prov_data = cn_prov_data[~cn_prov_data.isin(['境外输入'])].dropna(axis=0)
if '地区待确认' in cn_prov_data['name'].values.tolist():
    cn_prov_data = cn_prov_data[~cn_prov_data.isin(['地区待确认'])].dropna(axis=0)
if '外地来京' in cn_prov_data['name'].values.tolist():
    cn_prov_data = cn_prov_data[~cn_prov_data.isin(['外地来京'])].dropna(axis=0)

# 为了更直观看，将列名全部改为中文
cn_prov_data.columns = ['地区', '现有确诊', '累计确诊', '现有疑似', '累计死亡', '死亡率', '累计治愈', '治愈率']
cn_prov_data.drop(labels=[0], axis=0, inplace=True) # 删除第一行，第一行是具体省合计数据不需要显示
cn_prov_data

请输入要查看的省：北京


,地区,现有确诊,累计确诊,现有疑似,累计死亡,死亡率,累计治愈,治愈率
1,丰台,230,273,0,0,0.0,43,15.75
2,大兴,65,104,0,0,0.0,39,37.50
3,海淀,18,82,0,0,0.0,64,78.05
4,西城,6,59,0,0,0.0,53,89.83
5,东城,5,19,0,0,0.0,14,73.68
6,房山,4,20,0,0,0.0,16,80.00
7,门头沟,2,5,0,0,0.0,3,60.00
8,朝阳,2,77,0,0,0.0,75,97.40
9,通州,1,20,0,9,45.0,10,50.00
10,石景山,1,15,0,0,0.0,14,93.33


In [12]:
(
    Geo()
    .add_schema(maptype=f'{prov}') # 使用中国地图 
    .add(
        series_name='现有确诊', 
        data_pair=[(i, j) for i, j in zip(cn_prov_data['地区'], cn_prov_data['现有确诊'])], # data_pair是放数据的地方，类别和属性值
        label_opts=opts.LabelOpts(is_show=False),
        type_=ChartType.EFFECT_SCATTER # 图的类型为特效散点图
    ) # 也可以隐藏地图上的文字显示，不过悬浮点上的时候还是可以显示的 
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f'{prov}现有确诊'),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=False) # 视觉印象配置项 is_piecewise=True就不是连续的颜色了，而是拆分成块了
    )
).render_notebook()

f:\python\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


In [13]:
(
    Geo()
    .add_schema(maptype=f'{prov}') # 使用中国地图 
    .add(
        series_name='累计确诊', 
        data_pair=[(i, j) for i, j in zip(cn_prov_data['地区'], cn_prov_data['累计确诊'])], # data_pair是放数据的地方，类别和属性值
        label_opts=opts.LabelOpts(is_show=False),
        type_=ChartType.EFFECT_SCATTER # 图的类型为特效散点图
    ) # 也可以隐藏地图上的文字显示，不过悬浮点上的时候还是可以显示的 
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f'{prov}累计确诊'),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=False) # 视觉印象配置项 is_piecewise=True就不是连续的颜色了，而是拆分成块了
    )
).render_notebook()

f:\python\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


In [14]:
(
    Geo()
    .add_schema(maptype=f'{prov}') # 使用中国地图 
    .add(
        series_name='累计治愈', 
        data_pair=[(i, j) for i, j in zip(cn_prov_data['地区'], cn_prov_data['累计治愈'])], # data_pair是放数据的地方，类别和属性值
        label_opts=opts.LabelOpts(is_show=False),
        type_=ChartType.EFFECT_SCATTER # 图的类型为特效散点图
    ) # 也可以隐藏地图上的文字显示，不过悬浮点上的时候还是可以显示的 
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f'{prov}累计治愈'),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=False) # 视觉印象配置项 is_piecewise=True就不是连续的颜色了，而是拆分成块了
    )
).render_notebook()

f:\python\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


In [15]:
# 各省治愈率死亡率可视化
(
    Bar(init_opts=opts.InitOpts())
    .add_xaxis([i for i in cn_prov_data['地区']])
    .add_yaxis(series_name='死亡率', y_axis=[i for i in cn_prov_data['死亡率']], color='green', stack='stack1')
    .add_yaxis(series_name='治愈率', y_axis=[i for i in cn_prov_data['治愈率']], color='grey', stack='stack1')
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f"{prov}治愈率和死亡率"),
        tooltip_opts=opts.TooltipOpts(trigger='axis', axis_pointer_type='cross'),
        xaxis_opts=opts.AxisOpts(name='地区', axislabel_opts={"interval":"0"}),
        yaxis_opts=opts.AxisOpts(name='%', splitline_opts=opts.SplitLineOpts(is_show=True))
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
).reversal_axis().render_notebook()

f:\python\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


## 各省境外输入可视化

In [16]:
provinces_input_list = []
provinces = ['安徽', '澳门', '香港', '北京', '福建', '甘肃', '广东', '广西', '贵州', '海南', '河北', '河南', '黑龙江', '湖北', '湖南', '吉林', '江苏', '江西', '辽宁', '内蒙古', '宁夏', '青海', '山东', '山西', '陕西', '上海', '四川', '台湾', '天津', '西藏', '香港', '新疆', '云南', '浙江', '重庆']
for p in provinces:
    prov_data = pd.read_csv(filepath_cn+f"各省具体数据/{p}.csv", encoding='GBK')
    prov_data.columns = ['地区', '现有确诊', '累计确诊', '现有疑似', '累计死亡', '死亡率', '累计治愈', '治愈率']

    if '境外输入' in prov_data['地区'].values.tolist(): # 不一定所有的省都有境外输入
        input_data = prov_data.loc[prov_data['地区']=='境外输入', :].values.tolist()[0]
        input_data[0] = p # 将第一个字段原本的 境外输入 改成 具体的省份名称
        provinces_input_list.append(input_data)

# 生成DataFrame数据
provinces_input_data = pd.DataFrame(provinces_input_list, columns=['地区', '现有确诊', '累计确诊', '现有疑似', '累计死亡', '死亡率', '累计治愈', '治愈率'])
provinces_input_data

,地区,现有确诊,累计确诊,现有疑似,累计死亡,死亡率,累计治愈,治愈率
0,安徽,0,1,0,0,0.0,1,100.00
1,北京,0,174,0,0,0.0,174,100.00
2,福建,2,68,0,0,0.0,66,97.06
3,甘肃,0,75,0,0,0.0,75,100.00
4,广东,16,266,0,0,0.0,250,93.98
5,广西,1,3,0,0,0.0,2,66.67
6,贵州,0,1,0,0,0.0,1,100.00
7,海南,0,2,0,0,0.0,2,100.00
8,河北,0,10,0,0,0.0,10,100.00
9,河南,0,3,0,0,0.0,3,100.00


In [17]:
(
    Geo()
    .add_schema(maptype="china") # 使用中国地图 
    .add(
        series_name='现有确诊', 
        data_pair=[(i, j) for i, j in zip(provinces_input_data['地区'], provinces_input_data['现有确诊'])], # data_pair是放数据的地方，类别和属性值
        label_opts=opts.LabelOpts(is_show=False),
        type_=ChartType.EFFECT_SCATTER # 图的类型为特效散点图
    ) # 也可以隐藏地图上的文字显示，不过悬浮点上的时候还是可以显示的 
    .set_global_opts(
        title_opts=opts.TitleOpts(title="中国境外输入 现有确诊"),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=False) # 视觉印象配置项 is_piecewise=True就不是连续的颜色了，而是拆分成块了
    )
).render_notebook()

f:\python\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


In [18]:
(
    Geo()
    .add_schema(maptype="china") # 使用中国地图 
    .add(
        series_name='累计确诊', 
        data_pair=[(i, j) for i, j in zip(provinces_input_data['地区'], provinces_input_data['累计确诊'])], # data_pair是放数据的地方，类别和属性值
        label_opts=opts.LabelOpts(is_show=False),
        type_=ChartType.EFFECT_SCATTER # 图的类型为特效散点图
    ) # 也可以隐藏地图上的文字显示，不过悬浮点上的时候还是可以显示的 
    .set_global_opts(
        title_opts=opts.TitleOpts(title="中国境外输入 累计确诊"),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=False) # 视觉印象配置项 is_piecewise=True就不是连续的颜色了，而是拆分成块了
    )
).render_notebook()

f:\python\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


In [19]:
(
    Geo()
    .add_schema(maptype="china") # 使用中国地图 
    .add(
        series_name='累计治愈', 
        data_pair=[(i, j) for i, j in zip(provinces_input_data['地区'], provinces_input_data['累计治愈'])], # data_pair是放数据的地方，类别和属性值
        label_opts=opts.LabelOpts(is_show=False),
        type_=ChartType.EFFECT_SCATTER # 图的类型为特效散点图
    ) # 也可以隐藏地图上的文字显示，不过悬浮点上的时候还是可以显示的 
    .set_global_opts(
        title_opts=opts.TitleOpts(title="中国境外输入 累计治愈"),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=False) # 视觉印象配置项 is_piecewise=True就不是连续的颜色了，而是拆分成块了
    )
).render_notebook()

f:\python\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
